# Single Excitation Configuration Interaction

In this notebook, we will be studying the effect of single excitation configuration interaction on the resuls obtained from constrained unrestricted hartree-fock. We will use the package psi4 for reference data and some basic methods. The UHF and CUHF code will come from the compChem package. We will begin this discussion with a theoretical approach to CIS and on the implementation in a normal UHF system. We will then proceed to extrapolate these findings to CUHF. The theory will be developed for RHF and then implemented for UHF, with the nessecary adjustments.

### Theoretical approach, findings and questions


When trying to make sanes of the theory, we need to make some findings. The sources here are 
- Szabo & Ostlund, _Modern Quantum Chemistry_
- Sherrill, _Derivation of the Configuration Interaction Singles (CIS) Method for Various Single Determinant References and Extensions to Include Selected Double Substitutions (XCIS)_
- the psi4  numpy tutorials, chapter 9


At first, let us take a look at the hamiltonian matrix in CIS. We know that we can express the wave function as Equation (1).
\begin{equation*} \tag{1} 
\Phi = c_0\Psi_0 + \sum_{ia} c^a_i\Phi^a_i
\end{equation*}
$\Psi_0$ is the normal HF wavefuntion. This function will be a better approximation for the exact wavefunction. However we will need to solve the schrödinger equation again, now using this new function. We will need to make a new hamiltonian, including all excited states as well as the ground state. There are a lot of possible combinations and one can imagine that it will be difficult to express let alone solve this matrix. However, we could also change our basis. When we use the ground state and the excited states as a basis, we find the following matrix (2).
\begin{equation*}\tag{2}
\hat{H} = \begin{bmatrix}\langle \Psi_0 |\hat{H}|\Psi_0 \rangle & 0 & 0 & \dots \\  
0 & \langle \Phi_i^a |\hat{H}|\Phi_i^a \rangle & \langle \Phi_i^b |\hat{H}|\Phi_i^a \rangle & \dots\\
0 & \langle \Phi_i^a |\hat{H}|\Phi_i^b \rangle & \langle \Phi_i^b |\hat{H}|\Phi_i^b \rangle & \dots\\
\end{bmatrix}
\end{equation*}
Overall the dimensions of this matrix will be the amount of occupied orbitals ($o$) times the amount of unoccupied orbitals ($u$), since for every occupied orbital $o$ there will be $u$ possible ways to create an excited state. However, note that in the represention that is being used here, $\Psi_0$ is also an eigenfunction of the hamiltonian, so we added an extra row and an extra collumn. The elements in the added rows and collums are all zero in accordance to Brillouin's theorem, which states that if $\Psi_0$ is a self consistent Hartree-Fock wave function, the overlap elements of the hamiltonian are zero for any single excited state. This means that the hamiltonian is block diagonal, with one very large block. Nevertheless, this is a step forward from it not being block diagonal. We can also pretty easily express these matrix elements. This can be seen in Equation (3).
\begin{equation*}\tag{3}
\langle \Phi_i^a |\hat{H}| \Phi_j^b \rangle = E_0\delta_{ij}\delta{ab} + F_{ab}\delta_{ij} - F_{ij}\delta_{ab} + \langle aj|| ib \rangle
\end{equation*}
Now that we have an expression for our hamiltonian, we can solve the eigenequations for it. The only thing left to figure out is what basis this hamiltonian is given in. Since it is made up out of the ground state and the excited states, these should all be eigenfunctions of the hamiltonian itself, wo the basis used is MO basis. We could start using the eigenfunctions of the fock operator at scf conditions (which is off course needed for Brillouin's theorem to hold) and start iterating from there. So we would calculate the eigenfunctions and the energy, use these to create a new operator and repeat. Alternatively we could substract $E_0$ from it, so that we get the change in energy with respect to the orginal HF energy. An additional note we can make is that $F_{ij} = \delta_{ij}\epsilon_i$ at scf conditions, since the fock matrix should be diagonal there. This will simplify Equation (4) to Equation (5), in which we also substracted $E_0$.
\begin{equation*}\tag{5}
\langle \Phi_i^a |\hat{H} - E_0| \Phi_j^b \rangle =  (\epsilon_a - \epsilon_i)\delta_{ab}\delta_{ij} + \langle aj|| ib \rangle
\end{equation*}

Questions at this point:
- Sherrill, equation (7): where does the - sign come from?
- Sherrill, equation (12): should this be $\epsilon_a - \epsilon_i$?

### Codeblock 1: expressing the CIS Hamiltonian

Now the problem at hand is the implementation. We need to find a way to get this concept into some kind of code for UHF. Let us begin with the stuf we will act upon. We will need to run the iteration for normal UHF and then use the matrices we get from here to implement CIS. When we calculate the eigenvalues of the hamiltonian, we will also get a coefficient matrix which expresses the eigenfunctions in MO basis. This can then be used to calculate the energy according to Equation (4).
\begin{equation*}\tag{4}
E_{CIS} = E_0 + 2\sum_{ia}c_0c_i^aF_{ia} + \sum_{iab} c_i^ac_i^bF_{ab} - \sum_{ija}c_i^ac_j^bF_{ij} + \sum_{ijab}c_i^ac_j^b \langle aj||ib \rangle
\end{equation*}
Note that the coefficients are known. Here we have a way to calculate the energy starting from the HF energy and adding some terms in. The only problem we face now is what matrices to use. The Fock matrices in the equation are MO Fock matrices. But does this equation hold for UHF? If not, how can we make it so? 